Настраиваем ngrok

In [1]:
import os
!pip install pyngrok
!pip install python-dotenv
from pyngrok import ngrok
from dotenv import load_dotenv

load_dotenv()
port = 5000

ngrok_token = os.getenv('NGROK_TOKEN')

#Setting an auth token allows us to open multiple tunnels at the same time
ngrok.set_auth_token(ngrok_token)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

     |████████████████████████████████| 745 kB 5.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=494dc5370ab4a439892c517a6bc7058409849dd13590e4cff8c2e6a5ed595c08
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
 * ngrok tunnel "http://0530-146-148-90-100.ngrok.io" -> "http://127.0.0.1:5000"


In [2]:
import requests
from flask import Flask, request
import threading
from bot import *

app = Flask(__name__)
token = os.getenv('BOT_TOKEN')
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = "https" + public_url.split('http')[1]
print(app.config["BASE_URL"])
bot = Bot(token, app.config["BASE_URL"])

@app.route("/", methods=["GET", "POST"])
def receive_update():
    if request.method == "POST":
        print(request.json)
        chat_id = request.json["message"]["chat"]["id"]
        photos = request.json["message"].get("photo")
        if photos:
          photo = photos[-1]
          downloaded_photo = bot.get_photo(photo['file_id'], public_url)
          downloaded_photo.save('./test.png')
          bot.send_message(chat_id, "nom-nom photo")
        else:
            print(request.json["message"]["text"])
            bot.send_message(chat_id, request.json["message"]["text"])
    return request.json["message"]["text"]

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
